In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install efficientnet_pytorch torchtoolbox

     |████████████████████████████████| 55 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 869 kB 7.7 MB/s eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=08086c28bbdaa02005f9c51b7a765e8fadddbc0ad154a923066782ff62311535
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for lmdb: filename=lmdb-0.98-cp37-cp37m-linux_x86_64.whl size=273152 sha256=52cc657e79421a9a81f2ea4541fd7eb1ba7c5aaff2b2782899dfc3ff5e1e5b94
  Stored in directory: /root/.cache/pip/wheels/9e/24/96/783d4dddcf63e3f8cc92db8b3af3c70cf6d76398bff77f1d5e
Successfully built efficientnet-pytorch lmdb


In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  25447      0 --:--:-- --:--:-- --:--:-- 25447
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Found existing installation: torch 1.5.0
Uninstalling torch-1.5.0:
  Successfully uninstalled torch-1.5.0
Found existing installation: torchvision 0.6.0a0+35d732a
Uninstalling torchvision-0.6.0a0+35d732a:
Done updating TPU runtime
  Successfully uninstalled torchvision-0.6.0a0+35d732a
Copying gs://tpu-pytorch/wheels/torch-nightly-cp37-cp37m-linux_x86_64.whl...
- [1 files][108.5 MiB/108.5 MiB]                                                
Operation completed over 1 objects/108.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl...
- [1 files][124.1 MiB/124.1 MiB]                                    

In [3]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau , CyclicLR
from torchvision import transforms

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold

from efficientnet_pytorch import EfficientNet
#import torch_xla.core.xla_model as xm

from joblib import Parallel, delayed

import os 
import gc
from tqdm.autonotebook import tqdm

import cv2
import datetime
import random
import warnings
import time

from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_csv = pd.read_csv('../input/jpeg-melanoma-384x384/train.csv')
test_csv = pd.read_csv('../input/jpeg-melanoma-384x384/test.csv')

In [6]:
#Removing all the data with NaN
train_1 = train_csv.loc[pd.notnull(train_csv['sex'])]
train_2 = train_1.loc[pd.notnull(train_csv['age_approx'])]
train_3 = train_2.loc[pd.notnull(train_csv['anatom_site_general_challenge'])]

In [7]:
site = pd.get_dummies(train_3['anatom_site_general_challenge'], prefix = 'site')

In [8]:
train_3['sex'] = train_3['sex'].map({'male':1, 'female':0 })
test_csv['sex'] = test_csv['sex'].map({'male':1, 'female': 0})

In [9]:
train_3  = pd.concat([train_3 , site] , axis=1)

In [10]:
meta_features = ['sex', 'age_approx']  + [ f for f in site.columns]

In [11]:
site_test = pd.get_dummies(test_csv['anatom_site_general_challenge'], prefix= 'site')
test_csv = pd.concat([test_csv, site_test], axis=1)

In [12]:
test_csv.drop(['anatom_site_general_challenge'], axis=1, inplace = True)
train_3.drop(['anatom_site_general_challenge'],axis=1, inplace=True)

In [13]:
train_3['sex'] = train_csv['sex']

In [14]:
train_3['sex'] = train_3['sex'].map({'male':1, 'female': 0})

In [15]:
meta_features = ['sex', 'age_approx'] + [c for c in train_3.columns if 'site' in c]

In [16]:
class RemoveHair:
  """
      Remove Hairs from images
  """
  def __init__(self):
    pass
    
  
    
  def __call__(self, image):

    grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(1, (17,17))

    blackhat = cv2.morphologyEx(grayscale, cv2.MORPH_BLACKHAT, kernel)

    _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)

    final_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)

    return final_image



In [17]:
class Microscope:

  def __init__(self, p):
    
    self.p = p

  def __call__(self, img):


    if random.random() < self.p:
      circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                          (img.shape[0]//2, img.shape[1]//2),
                          (random.randint(img.shape[0]//2 , img.shape[1]//2)),
                          (0,0,0),
                          -1)
      mask  = circle - 255
      img = np.multiply(img,mask)
    
    return img



In [18]:
class Net(nn.Module):

  def __init__(self, model, n_meta_features):
    super().__init__()
    
    self.model = model
    if 'EfficientNet' in str(model.__class__):
      self.model._fc = nn.Linear(in_features=1280, out_features=500, bias=True)
    
    self.meta = nn.Sequential(nn.Linear(n_meta_features, 500),
                              nn.BatchNorm1d(500),
                              nn.ReLU(),
                              nn.Dropout(p=0.2),
                              nn.Linear(500,250),
                              nn.BatchNorm1d(250),
                              nn.ReLU(),
                              nn.Dropout(p=0.2))
    
    self.output = nn.Linear(500 + 250, 1)

  def forward(self, inputs):

    x, meta = inputs
    cnn_features = self.model(x)
    meta_features = self.meta(meta)
    features  = torch.cat((cnn_features, meta_features), dim = 1)
    output = self.output(features)
    
    return output

In [19]:
class MelanomaDataset(Dataset):
  """
    Our Dataset for Melanoma Classification
    img_folder: Path to images directory
    meta_features: Additional Data features to be used
    df: Contains Meta_features
    transforms: Data Augmentation Techniques to be applied
  """
  def __init__(self, img_folder, df, meta_features = None, train = True, transforms = None ):
    
    super().__init__()

    self.img_folder = img_folder
    self.df = df
    self.meta_features = meta_features
    self.transforms = transforms
    self.train = train
    
  
  def __len__(self):
    return len(self.df)

  
  def __getitem__(self, index):

    img = os.path.join(self.img_folder , self.df.iloc[index]['image_name'] + '.jpg')
    meta = np.array(self.df.iloc[index][meta_features].values, dtype= np.float32)
    x = cv2.imread(img)

    if self.transforms:
      x = self.transforms(x)

    if self.train:
      y = self.df.iloc[index]['target']
      return (x,meta) , y
    else:
      return (x,meta)





In [20]:
train_aug = transforms.Compose([RemoveHair(),
                                Microscope(p=0.5),
                                transforms.ToPILImage(),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485,0.456,0.406], std = [0.229, 0.224, 0.225])])

test_aug = transforms.Compose([RemoveHair(),
                               transforms.ToPILImage(),
                               transforms.ToTensor(),
                               transforms.Normalize(mean=[0.485,0.456,0.406], std = [0.229, 0.224, 0.225])])


                                
                               



In [20]:
skf = GroupKFold(n_splits=5)
model = EfficientNet.from_pretrained('efficientnet-b1')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/checkpoints/efficientnet-b1-f1951068.pth



Loaded pretrained weights for efficientnet-b1


In [21]:
epochs = 5
model_path = 'model.pth'

oof = np.zeros((len(train_3), 1))
preds = torch.zeros((len(test_csv), 1), dtype = torch.float32, device=device)


In [21]:
test = MelanomaDataset(img_folder = '../input/jpeg-melanoma-384x384/test',
                      df = test_csv,
                      meta_features = meta_features,
                      train = False,
                      transforms = test_aug)

In [22]:
device

device(type='cuda')

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X = np.zeros(len(train_3)), 
                                                      y = train_3['target'], 
                                                      groups= train_3['patient_id'].tolist()), 
                                                      1):

  print('='*20, 'Fold', fold, '='*20 )
  model = EfficientNet.from_pretrained('efficientnet-b1')
  best_val = 0
  model = Net(model = model, n_meta_features = len(meta_features))

  model = model.to(device)

  optim = torch.optim.SGD(model.parameters(), lr=0.01)
  scheduler = ReduceLROnPlateau(optimizer= optim, mode = 'max', patience = 1, verbose=True, factor=0.2)
  criterion = nn.BCEWithLogitsLoss()
  #print('dataset')
  train = MelanomaDataset(df = train_3.iloc[train_idx].reset_index(drop=True),
                          img_folder = '../input/jpeg-melanoma-384x384/train',
                          meta_features = meta_features,
                          train = True,
                          transforms = train_aug)
  
  val = MelanomaDataset(df = train_3.iloc[val_idx].reset_index(drop=True),
                        img_folder = '../input/jpeg-melanoma-384x384/train',
                        meta_features = meta_features,
                        train = True,
                        transforms = test_aug)
  
  
  
  
  #print('loader')
  train_loader = DataLoader(dataset = train,
                            batch_size = 32,
                            shuffle = True,
                            num_workers = 2)

  
  val_loader =  DataLoader(dataset = val,
                            batch_size = 16,
                            shuffle = False,
                            num_workers = 2)
  
  
  test_loader = DataLoader(dataset = test,
                            batch_size = 16,
                            shuffle = False,
                            num_workers = 2)
  
  for epoch in range(epochs):

    start_time = time.time()
    correct = 0
    epoch_loss = 0
    model.train()
    #print('0')
    count=0
    for x,y in train_loader:
      
      #if count==0:
       #     print('yo')
        #    count = 1
      x[0] = torch.tensor(x[0], device = device, dtype= torch.float32)
      x[1] = torch.tensor(x[1], device = device, dtype= torch.float32)
      y    = torch.tensor( y, device = device, dtype= torch.float32)

      optim.zero_grad()
      z = model(x)

      loss = criterion(z, y.unsqueeze(1))
      loss.backward()
      
      optim.step()

      pred = torch.round(torch.sigmoid(z))
      correct += (pred.cpu() == y.cpu().unsqueeze(1)).sum().item()
      epoch_loss += loss.item()
    
    train_acc = correct / len(train_idx)
    model.eval()
    val_preds = torch.zeros((len(val_idx), 1), dtype = torch.float32, device = device)

    with torch.no_grad():

      for j, (x_val, y_val) in enumerate(val_loader):

        x_val[0] = torch.tensor(x_val[0], device = device, dtype= torch.float32)
        x_val[1] = torch.tensor(x_val[1], device = device, dtype= torch.float32)
        y_val    = torch.tensor(y_val   , device = device, dtype= torch.float32)
        
        z_val = model(x_val)
        val_pred = torch.sigmoid(z_val)
        val_preds[j*x_val[0].shape[0]:j*x_val[0].shape[0] + x_val[0].shape[0]] = val_pred
      #print('2')
      val_acc = accuracy_score( train_3.iloc[val_idx]['target'].values, torch.round(val_preds.cpu()))
      val_roc = roc_auc_score( train_3.iloc[val_idx]['target'].values, val_preds.cpu())

      print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Val acc: {:.3f} | Val roc_auc: {:.3f} | Training time: {}'.format(
            epoch + 1, 
            epoch_loss, 
            train_acc, 
            val_acc, 
            val_roc, 
            str(datetime.timedelta(seconds=time.time() - start_time))[:7]))

      scheduler.step(val_roc)

      if val_roc >= best_val:
        best_val = val_roc
        torch.save(model, model_path)
  
  model = torch.load(model_path)
  model.eval()
  val_preds = torch.zeros((len(val_idx), 1), dtype = torch.float32, device=device)
  #print('3')
  with torch.no_grad():

    for j, (x_val, y_val) in enumerate(val_loader):
      x_val[0] = torch.tensor(x_val[0], device = device, dtype= torch.float32)
      x_val[1] = torch.tensor(x_val[1], device = device, dtype= torch.float32)
      y_val    = torch.tensor(y_val   , device = device, dtype= torch.float32)

      z_val = model(x_val)
      val_pred = torch.sigmoid(z_val)
      val_preds[j*x_val[0].shape[0]:j*x_val[0].shape[0] + x_val[0].shape[0]] = val_pred
    oof[val_idx] = val_preds.cpu().numpy()
    #print('4')
    for i, x_test in enumerate(test_loader):
      x_test[0] = torch.tensor(x_test[0], device = device, dtype = torch.float32)
      x_test[1] = torch.tensor(x_test[1], device = device, dtype = torch.float32)
      z_test    = model(x_test)
      z_test    = torch.sigmoid(z_test)
      z_test = torch.sigmoid(z_test)
      preds[i*x_test[0].shape[0]:i*x_test[0].shape[0] + x_test[0].shape[0]] += z_test

  del train, val, train_loader, val_loader, x, y, x_val, y_val
  gc.collect

preds /= skf.n_splits


In [22]:
preds1 = torch.zeros((len(test_csv), 1), dtype = torch.float32, device=device)

In [23]:
m = torch.load('../input/model-1/model.pth')

In [29]:

m.eval()
test_loader1 = DataLoader(dataset = test,
                         batch_size = 16,
                         shuffle = False,
                         num_workers = 2)
with torch.no_grad():
    for i, x_test in enumerate(test_loader1):

          x_test[0] = torch.tensor(x_test[0], device = device, dtype = torch.float32)
          x_test[1] = torch.tensor(x_test[1], device = device, dtype = torch.float32)
          z_test    = m(x_test)
          z_test    = torch.sigmoid(z_test)
          z_test = torch.sigmoid(z_test)
          preds1[i*x_test[0].shape[0]:i*x_test[0].shape[0] + x_test[0].shape[0]] += z_test
      
